In [1]:
import sympy as sp
import time
from sympy import Derivative as D
from utils.symbolic import (infinitesimals_generator, higher_infinitesimals_generator,
                            group_operator, der_relabel, deriv_infts, subs_new_vars, Diff,
                           sym_det_eqn)
from utils.algebra import (str_to_dict, get_common_factors, get_det_eqns, str_eqn_to_dict_eqn, 
                            simplify_redundant_eqn)

In [2]:
x, y, z, t = sp.symbols('x y z t')
phi = sp.Function('phi')(x, y, z, t)
psi = sp.Function('psi')(x, y, z, t)
b = sp.Function('b')(x, y, z, t)
w = sp.Function('w')(x, y, z, t)
sigma, gamma = sp.symbols('sigma gamma')
list_indep = [x, y, z, t]
list_dep = [phi, psi, b, w]
list_var = list_indep + list_dep
list_cte = [sigma, gamma]
list_all = list_cte + list_indep + list_dep
F =  D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) + sigma*(D(psi, x)*(D(psi, x, x, y) + D(psi, y, y, y))-
        D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))) + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) - gamma*(D(b,x,x) + D(b, y, y))
F_rules_array = {D(psi, x, x, y): 
        - (sigma*D(psi, x))**(-1)*(D(b, t) + D(b, x)*D(phi, y) - D(b, y)*D(phi, x) 
        + D(psi, x)*D(w, y) - D(psi, y)*D(w, x) 
        - gamma*(D(b,x,x) + D(b, y, y)))
        +(D(psi, x))**(-1)*D(psi, y)*(D(psi, x, x, x) + D(psi, y, y, x))
        - D(psi, y, y, y)}
order = 3
F.expand()

-gamma*Derivative(b(x, y, z, t), (x, 2)) - gamma*Derivative(b(x, y, z, t), (y, 2)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (y, 3)) + sigma*Derivative(psi(x, y, z, t), x)*Derivative(psi(x, y, z, t), (x, 2), y) - sigma*Derivative(psi(x, y, z, t), (x, 3))*Derivative(psi(x, y, z, t), y) - sigma*Derivative(psi(x, y, z, t), y)*Derivative(psi(x, y, z, t), (y, 2), x) + Derivative(b(x, y, z, t), t) + Derivative(b(x, y, z, t), x)*Derivative(phi(x, y, z, t), y) - Derivative(b(x, y, z, t), y)*Derivative(phi(x, y, z, t), x) + Derivative(psi(x, y, z, t), x)*Derivative(w(x, y, z, t), y) - Derivative(psi(x, y, z, t), y)*Derivative(w(x, y, z, t), x)

In [3]:
x, t, k = sp.symbols('x t kappa')
u = sp.Function('u')(x, t)
v = sp.Function('v')(x, t)
list_indep = [x, t]
list_dep = [u, v]
list_var = list_indep + list_dep
list_cte = [k]
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - D(u,x,x) +D(v,t) + x*v
F_rules_array = {D(u,x,x):D(u,t)+D(v,t)+x*v}
order = 2
F.expand()

x*v(x, t) + Derivative(u(x, t), t) - Derivative(u(x, t), (x, 2)) + Derivative(v(x, t), t)

In [2]:
x, t = sp.symbols('x t')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_var = list_indep + list_dep
list_cte = []
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - D(u,x,x)
F_rules_array = {D(u,x,x):D(u,t)}
order = 2
F.expand()

Derivative(u(x, t), t) - Derivative(u(x, t), (x, 2))

In [2]:
x, t, k = sp.symbols('x t k')
u = sp.Function('u')(x, t)
list_indep = [x, t]
list_dep = [u]
list_var = list_indep + list_dep
list_cte = [k]
list_all = list_cte + list_indep + list_dep
F =  D(u,t) - k*D(u,x,x)
F_rules_array = {D(u,x,x):1/k*D(u,t)}
order = 2
F.expand()

-k*Derivative(u(x, t), (x, 2)) + Derivative(u(x, t), t)

In [3]:
t = time.time()
infts = infinitesimals_generator(list_indep, list_dep)
n_indep = len(list_indep)
n_dep = len(list_dep)
infts_ind = infts[0:n_indep]
infts_dep = infts[n_indep:(n_dep + n_indep )]
elapsed = time.time() - t
print(elapsed)

0.001994609832763672


In [4]:
t = time.time()
inft_derivatives, dep_vars_derivatives = higher_infinitesimals_generator(infts_ind, infts_dep,
                                                                            order, list_indep, list_dep)
infts_dummy = [item for sublist in inft_derivatives for item in sublist]
dep_vars_derivatives = [item for sublist in dep_vars_derivatives for item in sublist]
infts = infts + infts_dummy
elapsed = time.time() - t
print(elapsed)

0.38218188285827637


In [5]:
t = time.time()
F, deriv_names = der_relabel(dep_vars_derivatives, F)

vars_and_derivatives = list_indep + list_dep + deriv_names
elapsed = time.time() - t
print(elapsed)

0.0009975433349609375


In [6]:
t = time.time()
XF = group_operator(F, vars_and_derivatives, infts) # Es la que mas se tarda
elapsed = time.time() - t
print(elapsed)

1.4771103858947754


In [7]:
t = time.time()
XF = subs_new_vars(dep_vars_derivatives, deriv_names, XF)
elapsed = time.time() - t
print(elapsed)

0.0029926300048828125


In [8]:
t = time.time()
XF = sp.simplify(XF.subs(F_rules_array))
elapsed = time.time() - t
print(elapsed)

0.4251976013183594


In [9]:
str(XF.expand()).replace(" ","")

'-k*Derivative(eta^u(x,t,u(x,t)),(u(x,t),2))*Derivative(u(x,t),x)**2+k*Derivative(u(x,t),t)*Derivative(u(x,t),x)**2*Derivative(xi^t(x,t,u(x,t)),(u(x,t),2))+k*Derivative(u(x,t),t)*Derivative(u(x,t),x)*Subs(Derivative(xi^t(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)+k*Derivative(u(x,t),t)*Derivative(u(x,t),x)*Subs(Derivative(xi^t(x,t,_xi),_xi,x),_xi,u(x,t))+k*Derivative(u(x,t),t)*Subs(Derivative(xi^t(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,x)+k*Derivative(u(x,t),x)**3*Derivative(xi^x(x,t,u(x,t)),(u(x,t),2))+k*Derivative(u(x,t),x)**2*Subs(Derivative(xi^x(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)+k*Derivative(u(x,t),x)**2*Subs(Derivative(xi^x(x,t,_xi),_xi,x),_xi,u(x,t))+2*k*Derivative(u(x,t),x)*Derivative(xi^t(x,t,u(x,t)),u(x,t))*Derivative(u(x,t),t,x)+k*Derivative(u(x,t),x)*Subs(Derivative(xi^x(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,x)-k*Derivative(u(x,t),x)*Subs(Derivative(eta^u(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)-k*Derivative(u(x,t),x)*Subs(Derivative(eta^u(x,t,_xi),_xi,x),_xi,u(x,t))+2*k*Derivative(u(x,

In [10]:
t = time.time()
empty_det_eqn = get_common_factors(XF, list_dep, list_indep, list_cte)
elapsed = time.time() - t
print(elapsed)

-Derivative(u(x,t),x)^2+Derivative(u(x,t),t)*Derivative(u(x,t),x)^2+Derivative(u(x,t),t)*Derivative(u(x,t),x)+Derivative(u(x,t),t)*Derivative(u(x,t),x)+Derivative(u(x,t),t)+Derivative(u(x,t),x)^3+Derivative(u(x,t),x)^2+Derivative(u(x,t),x)^2+Derivative(u(x,t),x)*Derivative(u(x,t),t,x)+Derivative(u(x,t),x)-Derivative(u(x,t),x)-Derivative(u(x,t),x)+Derivative(u(x,t),t,x)-+Derivative(u(x,t),t)*Derivative(u(x,t),x)-Derivative(u(x,t),t)+Derivative(u(x,t),t)-Derivative(u(x,t),x)+
0.005984783172607422


In [11]:
empty_det_eqn

{'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2': None,
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)': None,
 'Derivative(u(x,t),t)': None,
 'Derivative(u(x,t),x)^3': None,
 'Derivative(u(x,t),x)*Derivative(u(x,t),t,x)': None,
 'Derivative(u(x,t),t,x)': None,
 'Derivative(u(x,t),x)^2': None,
 'Derivative(u(x,t),x)': None}

In [12]:
t = time.time()
det_eqn = get_det_eqns(XF, empty_det_eqn)
elapsed = time.time() - t
print(elapsed)

0.005983829498291016


In [13]:
det_eqn

{'Derivative(u(x,t),t)*Derivative(u(x,t),x)^2': ['+k*Derivative(xit(x,t,u(x,t)),(u(x,t),2))'],
 'Derivative(u(x,t),t)*Derivative(u(x,t),x)': ['+k*Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)',
  '+k*Subs(Derivative(xit(x,t,_xi),_xi,x),_xi,u(x,t))',
  '+2*Derivative(xix(x,t,u(x,t)),u(x,t))'],
 'Derivative(u(x,t),t)': ['+k*Subs(Derivative(xit(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,x)',
  '-Subs(Derivative(xit(x,_xi_2,u(x,t)),_xi_2),_xi_2,t)',
  '+2*Subs(Derivative(xix(_xi_1,t,u(x,t)),_xi_1),_xi_1,x)'],
 'Derivative(u(x,t),x)^3': ['+k*Derivative(xix(x,t,u(x,t)),(u(x,t),2))'],
 'Derivative(u(x,t),x)*Derivative(u(x,t),t,x)': ['+2*k*Derivative(xit(x,t,u(x,t)),u(x,t))'],
 'Derivative(u(x,t),t,x)': ['+2*k*Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1),_xi_1,x)'],
 'Derivative(u(x,t),x)^2': ['-k*Derivative(etau(x,t,u(x,t)),(u(x,t),2))',
  '+k*Subs(Derivative(xix(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)',
  '+k*Subs(Derivative(xix(x,t,_xi),_xi,x),_xi,u(x,t))'],
 'Derivative(u(x,t),x)': ['+k*Su

In [19]:
t = time.time()
det_eqn = str_eqn_to_dict_eqn(det_eqn, list_var, list_all)
elapsed = time.time() - t
print(elapsed)

['+k*Derivative(xit(x,t,u(x,t)),(u(x,t),2))']
['+k*Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)', '+k*Subs(Derivative(xit(x,t,_xi),_xi,x),_xi,u(x,t))', '+2*Derivative(xix(x,t,u(x,t)),u(x,t))']
['+k*Subs(Derivative(xit(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,x)', '-Subs(Derivative(xit(x,_xi_2,u(x,t)),_xi_2),_xi_2,t)', '+2*Subs(Derivative(xix(_xi_1,t,u(x,t)),_xi_1),_xi_1,x)']
['+k*Derivative(xix(x,t,u(x,t)),(u(x,t),2))']
['+2*k*Derivative(xit(x,t,u(x,t)),u(x,t))']
['+2*k*Subs(Derivative(xit(_xi_1,t,u(x,t)),_xi_1),_xi_1,x)']
['-k*Derivative(etau(x,t,u(x,t)),(u(x,t),2))', '+k*Subs(Derivative(xix(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)', '+k*Subs(Derivative(xix(x,t,_xi),_xi,x),_xi,u(x,t))']
['+k*Subs(Derivative(xix(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,x)', '-k*Subs(Derivative(etau(_xi_1,t,u(x,t)),_xi_1,u(x,t)),_xi_1,x)', '-k*Subs(Derivative(etau(x,t,_xi),_xi,x),_xi,u(x,t))', '-Subs(Derivative(xix(x,_xi_2,u(x,t)),_xi_2),_xi_2,t)']
['-k*Subs(Derivative(etau(_xi_1,t,u(x,t)),(_xi_1,2)),_xi_1,

In [20]:
det_eqn

{0: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [0, 0, 2],
   'variable': 'xit'}],
 1: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [1, 0, 1],
   'variable': 'xit'},
  {'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [1, 0, 1],
   'variable': 'xit'},
  {'coefficient': 2,
   'constants': [0, 0, 0, 0],
   'derivatives': [0, 0, 1],
   'variable': 'xix'}],
 2: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [2, 0, 0],
   'variable': 'xit'},
  {'coefficient': -1,
   'constants': [0, 0, 0, 0],
   'derivatives': [0, 1, 0],
   'variable': 'xit'},
  {'coefficient': 2,
   'constants': [0, 0, 0, 0],
   'derivatives': [1, 0, 0],
   'variable': 'xix'}],
 3: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [0, 0, 2],
   'variable': 'xix'}],
 4: [{'coefficient': 2,
   'constants': [1, 0, 0, 0],
   'derivatives': [0, 0, 1],
   'variable': 'xit'}],
 5: [{'coefficient': 2,
   'constants': [1, 0, 0, 0]

In [21]:
t = time.time()
det_eqn = simplify_redundant_eqn(det_eqn)
elapsed = time.time() - t
print(elapsed)

0.0009975433349609375


In [22]:
det_eqn

{0: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [0, 0, 2],
   'variable': 'xit'}],
 1: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [1, 0, 1],
   'variable': 'xit'},
  {'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [1, 0, 1],
   'variable': 'xit'},
  {'coefficient': 2,
   'constants': [0, 0, 0, 0],
   'derivatives': [0, 0, 1],
   'variable': 'xix'}],
 2: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [2, 0, 0],
   'variable': 'xit'},
  {'coefficient': -1,
   'constants': [0, 0, 0, 0],
   'derivatives': [0, 1, 0],
   'variable': 'xit'},
  {'coefficient': 2,
   'constants': [0, 0, 0, 0],
   'derivatives': [1, 0, 0],
   'variable': 'xix'}],
 3: [{'coefficient': 1,
   'constants': [1, 0, 0, 0],
   'derivatives': [0, 0, 2],
   'variable': 'xix'}],
 4: [{'coefficient': 2,
   'constants': [1, 0, 0, 0],
   'derivatives': [0, 0, 1],
   'variable': 'xit'}],
 5: [{'coefficient': 2,
   'constants': [1, 0, 0, 0]

In [23]:
t = time.time()
sym_det_eqn(det_eqn, list_indep, list_dep, list_cte)
elapsed = time.time() - t
print(elapsed)

0.01746082305908203


In [24]:
sym_det_eqn(det_eqn, list_indep, list_dep, list_cte)

Matrix([
[1,                                 Eq(xi^(t)_u_u, 0)],
[2,                Eq(2*k*xi^(t)_x_u + 2*xi^(x)_u, 0)],
[3,       Eq(k*xi^(t)_x_x - xi^(t)_t + 2*xi^(x)_x, 0)],
[4,                                 Eq(xi^(x)_u_u, 0)],
[5,                                   Eq(xi^(t)_u, 0)],
[6,                                   Eq(xi^(t)_x, 0)],
[7,            Eq(-eta^(u)_u_u*k + 2*k*xi^(x)_x_u, 0)],
[8, Eq(-2*eta^(u)_x_u*k + k*xi^(x)_x_x - xi^(x)_t, 0)],
[9,                  Eq(eta^(u)_t - eta^(u)_x_x*k, 0)]])